In [ ]:
!pip install catboost

In [ ]:
from google.colab import drive
import pandas as pd

drive.mount('/content/drive', force_remount=True)

file_path = '/content/drive/MyDrive/project2/AI_agent_train_sepsis.csv'

data = pd.read_csv(file_path)

In [ ]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split, GroupKFold
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix, f1_score, precision_recall_curve
import lightgbm as lgb
import xgboost as xgb
from sklearn.ensemble import StackingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import re
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
if 'mortality_90d' in data.columns:
    value_counts = data['mortality_90d'].value_counts()
    print("mortality_90d 값 분포:")
    print(value_counts)
    print("\n비율:")
    print(data['mortality_90d'].value_counts(normalize=True) * 100, "%")
else:
    print("mortality_90d 컬럼이 데이터셋에 존재하지 않습니다.")

In [ ]:
def aggregate_patient_data(df):
    numerical_cols = [col for col in df.columns if col not in ['mortality_90d', 'icustayid', 'charttime', 'bloc']]

    agg_dict = {
        'mortality_90d': 'first',
    }

    for col in numerical_cols:
        if col == 'age':
            agg_dict[col] = 'first'
        else:
            agg_dict[col] = ['mean', 'min', 'max', 'std', 'last']

    agg_dict['charttime'] = 'count'

    agg_df = df.groupby('icustayid').agg(agg_dict)

    agg_df.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in agg_df.columns]

    return agg_df

patient_df = aggregate_patient_data(data)

def clean_feature_names(df):
    clean_df = df.copy()

    clean_columns = {}
    for col in clean_df.columns:
        new_col = re.sub(r'[^\w\s_]', '', col)
        new_col = re.sub(r'\s+', '_', new_col)
        clean_columns[col] = new_col

    clean_df = clean_df.rename(columns=clean_columns)

    return clean_df

patient_df = clean_feature_names(patient_df)

X = patient_df.drop('mortality_90d_first', axis=1)
y = patient_df['mortality_90d_first']


patient_ids = patient_df.index.unique().tolist()
np.random.seed(42)
np.random.shuffle(patient_ids)
train_size = int(len(patient_ids) * 0.8)
train_ids = patient_ids[:train_size]
test_ids = patient_ids[train_size:]

X_train = X.loc[train_ids]
X_test = X.loc[test_ids]
y_train = y.loc[train_ids]
y_test = y.loc[test_ids]

In [ ]:
def evaluate_model(model, X_test, y_test):
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    auc = roc_auc_score(y_test, y_pred_proba)
    precision_curve, recall_curve, thresholds = precision_recall_curve(y_test, y_pred_proba)
    f1_scores = 2 * precision_curve * recall_curve / (precision_curve + recall_curve + 1e-7)
    optimal_idx = np.argmax(f1_scores)
    optimal_threshold = thresholds[optimal_idx] if optimal_idx < len(thresholds) else 0.5

    y_pred = (y_pred_proba >= optimal_threshold).astype(int)

    optimal_precision = precision_curve[optimal_idx]
    optimal_recall = recall_curve[optimal_idx]
    optimal_f1 = f1_score(y_test, y_pred)

    return {
        'AUC': auc,
        'F1 Score': optimal_f1,
        'Optimal Threshold': optimal_threshold,
        'Predictions': y_pred,
        'Probabilities': y_pred_proba,
        'Optimal Precision': optimal_precision,
        'Optimal Recall': optimal_recall,
        'Precision Curve': precision_curve,
        'Recall Curve': recall_curve
    }

models = {
    'CatBoost': catboost_model,
    'LightGBM': lgb_model,
    'XGBoost': xgb_model,
    'RandomForest': rf_model,
    'Stacking': stacking_model
}

results = {}
for name, model in models.items():
    print(f"Evaluating {name} model...")
    results[name] = evaluate_model(model, X_test, y_test)

print("\nModel performance comparison (sorted by AUC):")
for name, metrics in sorted(results.items(), key=lambda x: x[1]['AUC'], reverse=True):
    print(f"{name}: Precision = {metrics['Optimal Precision']:.4f}, Recall = {metrics['Optimal Recall']:.4f}, F1 Score = {metrics['F1 Score']:.4f}, AUC = {metrics['AUC']:.4f}, Threshold = {metrics['Optimal Threshold']:.4f}")

print("\nModel performance comparison (sorted by F1 Score):")
for name, metrics in sorted(results.items(), key=lambda x: x[1]['F1 Score'], reverse=True):
    print(f"{name}: F1 Score = {metrics['F1 Score']:.4f}, AUC = {metrics['AUC']:.4f}, Threshold = {metrics['Optimal Threshold']:.4f}")

plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
auc_values = [metrics['AUC'] for name, metrics in results.items()]
model_names = list(results.keys())

sns.barplot(x=model_names, y=auc_values)
plt.title('AUC Comparison')
plt.ylim(0.7, 1.0)
plt.xticks(rotation=45)

plt.subplot(1, 2, 2)
f1_values = [metrics['F1 Score'] for name, metrics in results.items()]

sns.barplot(x=model_names, y=f1_values)
plt.title('F1 Score Comparison')
plt.ylim(0, 1.0)
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Save model

import pickle

models_to_save = {
    'catboost': catboost_model,
    'lightgbm': lgb_model,
    'xgboost': xgb_model,
    'randomforest': rf_model,
    'stacking': stacking_model
}

import os
os.makedirs('saved_models', exist_ok=True)


for name, model in models_to_save.items():
    with open(f'saved_models/{name}_model.pkl', 'wb') as f:
        pickle.dump(model, f)
    print(f"{name} 모델이 saved_models/{name}_model.pkl에 저장되었습니다.")


In [ ]:
models = {
    'CatBoost': catboost_model,
    'LightGBM': lgb_model,
    'XGBoost': xgb_model,
    'RandomForest': rf_model,
    'Stacking': stacking_model
}

import numpy as np
import matplotlib.pyplot as plt
import shap

if 'feature_names' not in globals():
    feature_names = X.columns.tolist()

def analyze_shap_values(models, X_test, feature_names):
    shap_values_dict = {}

    n_samples = min(1000, X_test.shape[0])
    X_sample = X_test.iloc[:n_samples]

    for name, model in models.items():
        print(f"Calculating SHAP values for {name} model...")
        try:
            explainer = shap.TreeExplainer(model)
            shap_values = explainer.shap_values(X_sample)

            if isinstance(shap_values, list):
                if len(shap_values) > 1:
                    shap_values = shap_values[1]

            shap_values_dict[name] = {
                'values': shap_values,
                'explainer': explainer,
                'data': X_sample
            }
            print(f"Successfully calculated SHAP values for {name}")
        except Exception as e:
            print(f"Error calculating SHAP values for {name}: {e}")

    return shap_values_dict

shap_values_dict = analyze_shap_values(models, X_test, feature_names)


In [ ]:
if shap_values_dict:
    for name, shap_data in shap_values_dict.items():
        plt.close('all')
        plt.figure(figsize=(12, 10))

        try:
            shap.summary_plot(
                shap_data['values'],
                shap_data['data'],
                feature_names=feature_names,
                max_display=10,
                show=False
            )
            plt.title(f'SHAP Feature Impact - {name}', fontsize=16)
            plt.tight_layout()
            plt.savefig(f'shap_feature_impact_{name}.png', bbox_inches='tight')
            plt.show()
            plt.close()
        except Exception as e:
            print(f"Error plotting SHAP summary for {name}: {e}")

        try:
            plt.figure(figsize=(20, 3))
            sample_idx = 0

            exp_value = shap_data['explainer'].expected_value
            if isinstance(exp_value, list):
                exp_value = exp_value[1]

            shap.force_plot(
                exp_value,
                shap_data['values'][sample_idx, :],
                shap_data['data'].iloc[sample_idx],
                feature_names=feature_names,
                matplotlib=True,
                show=False
            )
            plt.title(f'SHAP Force Plot - Sample {sample_idx}', fontsize=14)
            plt.tight_layout()
            plt.savefig(f'shap_force_plot_{name}_sample_{sample_idx}.png', bbox_inches='tight')
            plt.show()
            plt.close()
        except Exception as e:
            print(f"Error creating force plot: {e}")

        mean_abs_shap = np.abs(shap_data['values']).mean(axis=0)
        top_indices = np.argsort(-mean_abs_shap)[:5]

        for idx in top_indices:
            feature = feature_names[idx]
            plt.close('all')
            plt.figure(figsize=(10, 7))

            try:

                shap.dependence_plot(
                    idx,
                    shap_data['values'],
                    shap_data['data'],
                    feature_names=feature_names,
                    interaction_index=None,
                    show=False
                )
                plt.title(f'SHAP Dependence Plot - {feature}', fontsize=14)
                plt.tight_layout()
                plt.savefig(f'shap_dependence_{name}_{feature}.png', bbox_inches='tight')
                plt.show()
                plt.close()
            except Exception as e:
                print(f"Error creating dependence plot for {feature}: {e}")
else:
    print("No SHAP values were calculated. Check for errors in the previous steps.")